# 103590450 馬茂源 四資四

In [ ]:
import pandas as pd
import numpy as np
#from nltk.tokenize import word_tokenize, sent_tokenize
#from nltk import download
import time
import os
import sys
import string
import html
import hashlib
import re
from collections import defaultdict
import itertools

In [ ]:
#download('punkt')

In [ ]:
t0 = time.time()

In [ ]:
if not os.path.exists('result'):
    os.mkdir('result')
for i in range(1, 5):
    dir_ = 'result/task{}'.format(i)
    if not os.path.exists(dir_):
        os.mkdir(dir_)

In [ ]:
file_names = ['./data/reut2-{0:0>3}.sgm'.format(i) for i in range(22)]
file_names = [file_names[0]]

## (1) Given the Reuters-21578 dataset, please calculate all kshingles and output the set representation of the text dataset as a matrix. 

In [ ]:
def parser(file_name):
    with open(file_name, 'r', encoding='ISO-8859-1') as f:
        file = f.read()
    news = []
    start = 0
    for i in range(len(file)):
        if file[i:i+6] == '<BODY>':
            start = i+6
        elif file[i:i+7] == '</BODY>':
            n = file[start:i].replace('\n', ' ')
            n = n.replace('REUTER &#3;', '')
            news.append(n)
    return news

In [ ]:
news = []
for i in file_names:
    each_news = parser(i) 
    news.extend(each_news)

In [ ]:
len(news)

In [ ]:
news_data = pd.DataFrame(data=news, columns=['news'])

In [ ]:
news_data.head()

In [ ]:
def tokenizer(text):
    strip_chars = '.' + ' –…' + string.punctuation
    text = html.unescape(text)
    text = text.lower()
    text = text.strip(strip_chars)
    text = text.replace('reuter', '')
    text = re.sub(re.compile('<.*?>'), '', text)
    return re.findall(r'\w+', text)
    return text

In [ ]:
test_news = news_data['news'][1]

In [ ]:
#tokenizer(test_news)

In [ ]:
news_data['news_token'] = news_data.apply(lambda x: tokenizer(x['news']), axis=1)

In [ ]:
#news_data

In [ ]:
def k_shingle(text, k):
    string = ' '.join(text)
    shingles = set([])
    for i in range(len(string)-k + 1):
        shingles.add(string[i:i+k])
    return shingles

In [ ]:
news_data['news_token'] = news_data['news_token'].apply(lambda x: k_shingle(x, 5))

In [ ]:
shingles = set([])
for s in news_data['news_token'].values:
    shingles |= s

In [ ]:
N = len(shingles)
N

In [ ]:
shingles = list(shingles)

In [ ]:
def encode_shingles(row, shingles):
    return np.array([shingle in row for shingle in (shingles)], dtype='int') 

In [ ]:
news_data['shingles'] = news_data['news_token'].apply(encode_shingles, args=(shingles,))

In [ ]:
output_shingles = np.array(news_data['shingles'].tolist()).T#.astype('int')

In [ ]:
#np.savetxt("task_1.csv", output_shingles, delimiter=",",  fmt='%d')

## (2) Given the set representation, compute the minhash signatures of all documents using MapReduce.

In [ ]:
news_shingles = output_shingles

In [ ]:
def get_prime(greater_than):
    
    def is_prime(n):
        if n % 2 == 0 and n > 2: 
            return False
        return all(n % i for i in range(3, int(np.sqrt(n)) + 1, 2))
    is_p = False
    
    
    
    while not is_p:
        greater_than += 1
        is_p = is_prime(greater_than)
        
    return greater_than

In [ ]:
def get_hash_func_list(n, k=100):
    p = get_prime(n)
    func_list = []
    for a, b in zip(np.random.randint(0, n, size=k),
                   np.random.randint(0, n, size=k)):
        func_list.append(lambda x, a=a,b=b,p=p,n=n: ((a*x+b)%p)%n)
    return np.array(func_list)

In [ ]:
def one_pass_minhashing(shingles, k=100):
    n = shingles.shape[0]
    hash_list = get_hash_func_list(n, k=k)
    singnature = np.full((k, shingles.shape[1]), fill_value=np.inf)
    
    for i in range(n):
        hash_value = np.array([h(i) for h in hash_list])

        for j, c in enumerate(shingles[i, :] == 1):
            if c:
                mask = singnature[:, j] > hash_value
                singnature[:, j][mask] = hash_value[mask]
            
    return singnature

In [ ]:
test_input = np.array([[1,0,1,0],
                       [1,0,0,1],
                       [0,1,0,1],
                       [0,1,0,1],
                       [0,1,0,1],
                       [1,0,1,0],
                       [1,0,1,0]])
test_singnature = one_pass_minhashing(test_input, k=6)
test_singnature.astype('int')

In [ ]:
singnature = one_pass_minhashing(news_shingles, k=100)

In [ ]:
singnature.shape

In [ ]:
#np.savetxt("task_2.csv", singnature, delimiter=",",  fmt='%d')

## (3) Implement the LSH algorithm by MapReduce and output the resulting candidate pairs of similar documents.

In [ ]:
hashlib.sha512().block_size

In [ ]:
def LSH(singnature, b=2):
    buckets = [defaultdict(set) for i in range(b)]
    k = singnature.shape[0]
    r = k // b
    for i, doc in enumerate(singnature.T):
        x = np.array2string(doc.astype('int'), separator='', precision=0)
        for j, start_idx in enumerate(range(0, k, r)):
            #print(j, j+r)
            key = hashlib.sha512(x[start_idx:start_idx+r].encode()).hexdigest()
            buckets[j][key].add(i)
    return buckets

In [ ]:
def get_candidate(buckets):
    candidates = set([])
    for bucket in buckets:
        for k, item in bucket.items():
            if len(item) > 1:
                pairs = itertools.combinations(item, 2)
                for p in pairs:
                    candidates.add(p)

    return candidates

In [ ]:
# bucket = LSH(test_singnature, b=2)
# bucket

In [ ]:
buckets = LSH(singnature, b=20)
candidates = get_candidate(bucket)
candidates = list(candidates)

In [ ]:
len(candidates)

In [ ]:
new_candidates = []
for d_id_1, d_id_2 in candidates:
    intersection = np.logical_and(singnature[:, d_id_1],
                                  singnature[:, d_id_2])
    union = np.logical_or(singnature[:, d_id_1],
                          singnature[:, d_id_2])
    s = intersection.sum() / float(union.sum())
    if s >= 0.8:
        new_candidates.append((d_id_1, d_id_2))

In [ ]:
len(new_candidates)

In [ ]:
new_candidates[:10]

In [ ]:
news_data.loc[228]['news']

In [ ]:
news_data.loc[274]['news']

## (4) Implement K-nearest neighbor (KNN) search using LSH and compare its performance with linear search.

In [ ]:
print('cost:{:.3f} min'.format((time.time()-t0)/60))